Web Data Scraping

Website:http://www.landofbasketball.com/results/2014_2015_scores.htm

We need to scrape data and put into csv file

In [531]:
# We need information of teams, scores, date 
df

,1st_away,1st_home,2st_away,2st_home,3st_away,3st_home,4th_away,4th_home,away_team,home_team,month,season
first row,30,21,28,19,27,35,30,30,LAC,ATL,January,2017


In [3]:
#import the library used to query a website
import urllib2
#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup
import requests
import string
import pandas as pd
import re

In [15]:
year_time = ["2014_2015","2015_2016","2016_2017"]
wiki_origins =["http://www.landofbasketball.com/results/2014_2015_scores.htm","http://www.landofbasketball.com/results/2015_2016_scores.htm","http://www.landofbasketball.com/results/2016_2017_scores.htm"]
#whole_data = []
#data_fields =["season", "month", "away_team","home_team", "1st_away","1st_home","2st_away","2st_home","3st_away", "3st_home","4th_away","4th_home"]
output_data = pd.DataFrame()
for i in range(len(wiki_origins)):
    wiki_origin = wiki_origins[i]
    #print wiki_origin
    page_home = urllib2.urlopen(wiki_origin)
    #Parse the html in the 'page' variable, and store it in Beautiful Soup format
    soup_home = BeautifulSoup(page_home)
    soup_home.find_all('a')
    #Use function “prettify” to look at nested structure of HTML page
    month_list = soup_home.find_all(href=re.compile("^"+year_time[i]))
    month_list
    month_list_links = [a.attrs.get('href') for a in month_list]
    root_url = "http://www.landofbasketball.com/results/"
    second_link = [root_url + month_list_links[a] for a in range(len(month_list_links))]
    second_link = list(set(second_link))
    #'http://www.landofbasketball.com/results/2014_2015_may_scores.htm'
    for j in range(len(second_link)):
        wiki = second_link[j]
        print wiki
        page = urllib2.urlopen(wiki)
        #Parse the html in the 'page' variable, and store it in Beautiful Soup format
        soup = BeautifulSoup(page)
        #Use function “prettify” to look at nested structure of HTML page
        #soup.find_all(href=re.compile("box_scores"))
        #only need child websites starts with box_scores
        quarter_list = soup.find_all(href=re.compile("box_scores"))
        links = [a.attrs.get('href') for a in quarter_list]
        #links
        root_url = "http://www.landofbasketball.com"
        #print len(links)
        a = 0
        for a in range(len(links)):
            links[a] = links[a].replace("..", "") 
            a += 1
        #print links[0]
        quarter_2014 = [root_url + links[a] for a in range(len(links))]
        quarter_2014_deduplicate = list(set(quarter_2014))
        #we will get the website: http://www.landofbasketball.com/box_scores/2014/1028ORLNOP.htm
        for m in range(len(quarter_2014_deduplicate)):
            page = urllib2.urlopen(quarter_2014_deduplicate[m])
            #print page
            soup = BeautifulSoup(page)
            #find right tables
            score_table = soup.find("div", {"class": "rd-100-50 margen-b6 margen-b3"})
            if score_table is None:
                continue
            A = []
            for row in score_table.findAll("tr"):
                cells = row.findAll('td')
                cols = [ele.text.strip() for ele in cells]
                A.append([ele for ele in cols if ele])
            #print A
            #################
            score_table2 = soup.find("div", {"class": "rd-100-30 margen-b6 clearfix"})
            if score_table2 is None:
                continue
            A2= []
            for row in score_table2.findAll("p"):
                #print row
                text = row.text.strip()
                text = text.split(": ",1)[1]
                #print text
                A2.append(text)
            date = str(A2[0])
            month = date.split(" ",1)[0]
            #print month
            season = date.split(",",1)[1]
            #print season
            hometeam =  str(A2[1])
            #print hometeam
            score_table3 = soup.find("div", {"class": "rd-tbl-2"})
            A3 = []
            for row in score_table3.findAll("tr"):
                cells = row.findAll('td')
                cols = [ele.text.strip() for ele in cells]
                A3.append([ele for ele in cols if ele])
            away_team_temp= str(A3[0][0])
            #home_team_temp= str(A3[20][0])
            away_team = " ".join(re.findall("[a-zA-Z]+", away_team_temp))
            #home_team = " ".join(re.findall("[a-zA-Z]+", home_team_temp))
            d={}
            df=pd.DataFrame()
            if A[1][0].lower() in hometeam.lower():
                #whole_data.append([season,month,away_team,home_team,A[2][1],A[1][1],A[2][2],A[1][2],A[2][3],A[1][3],A[2][4],A[1][4]])
                d = {"season":season, "month":month, "away_team":away_team, "home_team":hometeam, "1st_away":A[2][1], "1st_home":A[1][1],"2st_away":A[2][2], "2st_home":A[1][2],"3st_away":A[2][3], "3st_home":A[1][3],"4th_away":A[2][4], "4th_home":A[1][4]}
                df = pd.DataFrame(data=d, index=["rows",])
                df.drop_duplicates()
                output_data = output_data.append(df)
                #df.to_csv(df["away_team"][0]+'_'+df["home_team"][0]+'_'+df["month"][0]+'_'+df["season"][0]+' .csv')
            else:
                #whole_data.append([season,month,away_team,home_team,A[1][1],A[2][1],A[1][2],A[2][2],A[1][3],A[2][3],A[1][4],A[2][4]])
                d = {"season":season, "month":month, "away_team":away_team, "home_team":hometeam, "1st_away":A[1][1], "1st_home":A[2][1],"2st_away":A[1][2], "2st_home":A[2][2],"3st_away":A[1][3], "3st_home":A[2][3],"4th_away":A[1][4], "4th_home":A[2][4]}
                df = pd.DataFrame(data=d, index=["rows",])
                df.drop_duplicates()
                output_data = output_data.append(df)
                #df.to_csv(df["away_team"][0]+'_'+df["home_team"][0]+'_'+df["month"][0]+'_'+df["season"][0]+' .csv')
            if len(output_data) == 0:
                print "output_data is none!"
print len(output_data)

http://www.landofbasketball.com/results/2014_2015_may_scores.htm
http://www.landofbasketball.com/results/2014_2015_mar_scores.htm
http://www.landofbasketball.com/results/2014_2015_jan_scores.htm
http://www.landofbasketball.com/results/2014_2015_apr_scores.htm
http://www.landofbasketball.com/results/2014_2015_jun_scores.htm
http://www.landofbasketball.com/results/2014_2015_oct_scores.htm
http://www.landofbasketball.com/results/2014_2015_scores_full.htm
http://www.landofbasketball.com/results/2014_2015_nov_scores.htm
http://www.landofbasketball.com/results/2014_2015_feb_scores.htm
http://www.landofbasketball.com/results/2014_2015_dec_scores.htm
http://www.landofbasketball.com/results/2015_2016_nov_scores.htm
http://www.landofbasketball.com/results/2015_2016_mar_scores.htm
http://www.landofbasketball.com/results/2015_2016_oct_scores.htm
http://www.landofbasketball.com/results/2015_2016_may_scores.htm
http://www.landofbasketball.com/results/2015_2016_jun_scores.htm
http://www.landofbasketb

In [18]:
output_data.to_csv('NBA_2014_2017_Data_1.csv') 